In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import scipy.signal as signal
import scipy.integrate as integrate
import matplotlib.pyplot as plt
from BinaryFileUnpack import BinaryFileUnpack
import os
import warnings

In [ ]:
def get_omega(start:float, end:float, Data:np.ndarray, obj:BinaryFileUnpack) -> tuple:
    fs = obj.fs
    s = int(start * fs)
    e = int(end * fs)
    # Don't use spectra method, use Lomb-Scargle method for deterministic frequencies
    omegas = np.linspace(2, 9, 2000)
    pgram = signal.lombscargle(obj.time[s:e], Data, omegas, precenter=True)
    power = 10*np.log10(pgram)
    
    max_ind = np.argmax(power)
    omega = omegas[max_ind]

    # Std. dev. (sigma) for omega
    # Use normal distribution with deciBel conversion with form
    # N(f) = N(fmax) - 10/ln(10) * ((f-fmax)/sigma)^2
    # A point is chosen 10 indices from fmax, such that
    # sigma = (10*res) * sqrt( (10/ln(10)) / (N(fmax) - N(fmax - 10*res)) )
    # res is the gap in frequency data
    res = omegas[1]-omegas[0]
    delta = 10*res
    sigma_omega = delta * np.sqrt( (10/np.log(10)) / (power[max_ind] - power[max_ind-10]) )
    
    return omega, sigma_omega

# Specify base pressures and H based on conduit diameter and sensor index
def spec_params(cond_diam, sens_ind, path, x):
    if sens_ind == 2:
        if cond_diam == 1:
            pres = 1.2589916544823685 # bar
            pres_std = 5.423258097482985e-05 # bar
            H = 58 # cm
        elif cond_diam == 2:
            pres = 1.3079672852169744 # bar
            pres_std = 0.00014948905833631553 # bar
            H = 46 # cm
        # Correction due to atmospheric calibration
        pres = pres*0.874466484 - 0.0786998490
        pres_std *= 0.874466484
    elif sens_ind == 0:
        # Use extracted values for sensor 1 calibration
        df_cal = pd.read_csv(f"{path}/../../sensor1_calibration.csv")
        pres = df_cal.iloc[0][0] + x*df_cal.iloc[1][0]
        pres_std = np.sqrt(df_cal.iloc[0][1]**2 + x*df_cal.iloc[1][1]**2)
        
        if cond_diam == 1:
            H = 58 # cm
        elif cond_diam == 2:
            H = 46
        # Correction due to atmospheric calibration
        pres = pres*1.05029088 - 0.298257335
        pres_std *= 1.05029088
    
    return pres, pres_std, H

The following data is for 1-inch diameter conduit.

In [ ]:
path = r"C:\Users\akyap\OneDrive\Documents\Academics\Research\LDEO Geysers\files\tests\hot-water-1in"
X38cm = [
    (92.6, 112.4, 2),
]

X51cm_0 = [
    (26.7, 35, 2),
    # Oscillation from ~ 51 to 55 sec unuseable
]

X51cm_1 = [
    (11.5, 24.5, 2),
    (49.4, 59.8, 2),
    (71.8, 82.0, 2),
    (97, 109, 2),
    (137, 146.9, 2)
]

X51cm_2 = [
    (31.2, 45.2, 2),
    (56, 67.7, 2),
    (56, 67, 2),
    (77.6, 90.3, 2),
    (111.6, 129, 2),
    (139.5, 149.5, 2),
    (174.75, 186, 2)
]

X60cm = [
    (14.9, 28.0, 2),
    (53.7, 64.2, 2),
    (77.5, 90, 2),
    (133.7, 147.2, 2),
    (175, 188, 2),
    (233, 250.5, 2),
]

X66cm = [
    (24.6, 38.3, 2),
    (60.8, 72.0, 2),
    (93.15, 110.6, 2),
    (130, 140.2, 2),
    (158, 173, 2),
    (215, 228.2, 2),
    (261.5, 274, 2),
    (291, 302.2, 2),
    (322, 333, 2)
]

X71cm = [
    (23.7, 34.8, 2),
    (54, 64.3, 2),
    (72.8, 82.6, 2),
    (98.8, 108.5, 2),
    (115, 123, 2),
    (153.4, 163.0, 2),
    (189.5, 200, 2),
]

tests_1inch = {
    'X38cm':(X38cm, path+r'\X38cm\HotWaterOsc_X38cm_YNA-20220805-18-25-16.bin'),
    'X51cm_0':(X51cm_0, path+r'\X51cm\HotWaterOsc_X51cm_YNA-20220805-18-49-15.bin'),
    'X51cm_1':(X51cm_1, path+r'\X51cm\HotWaterOsc_X51cm_YNA-20220805-18-52-04.bin'),
    'X51cm_2':(X51cm_2, path+r'\X51cm\HotWaterOsc_X51cm_YNA-20220805-18-56-47.bin'),
    'X60cm':(X60cm, path+r'\X60cm\HotWaterOsc_X60cm_YNA-20220805-19-17-07.bin'),
    'X66cm':(X66cm, path+r'\X66cm\HotWaterOsc_X66cm_YNA-20220805-19-29-43.bin'),
    'X71cm':(X71cm, path+r'\X71cm\HotWaterOsc_X71cm_YNA-20220805-19-45-56.bin')
}

The following data is for 2-inch diameter conduit.

In [ ]:
path = r"C:\Users\akyap\OneDrive\Documents\Academics\Research\LDEO Geysers\files\tests\hot-water-2in"
X49cm = [
    (39.6,   50.0, 2),
    (60.2,   70.0, 2),
    (80.1,   90.0, 2),
    (107.7, 116.0, 2),
    (137.8, 145.0, 0),
    (155.2, 163.2, 0),
    (309.8, 318.0, 2),
    (331.7, 340.0, 2),
    (351.5, 362.0, 2),
    (370.0, 380.0, 2),
    (397.7, 408.0, 0),
]

X64cm = [
    (54.4,   64.0, 0),
    (77.1,   83.0, 2),
    (109.0, 115.0, 2),
    (183.1, 190.0, 2),
    (213.0, 220.0, 2),
    (229.5, 235.6, 2),
    (242.0, 247.8, 2),
    (261.0, 270.2, 2),
]

tests_2inch = {
    'X49cm':(X49cm, path+r'\HotWater_X49cm-20221104-20-34-41.bin'),
    'X64cm':(X64cm, path+r'\HotWater_X64cm-20221104-20-23-34.bin')
}

In [ ]:
# Defining global physical parameters
Hb = 76  # cm
t  = 8.1 # cm

# tests dictionary should be in format of 
# tests = { 'X<x-value>':(<data_arr>, filepath), ... }

cond_diam = 1
if cond_diam == 1:
    tests = tests_1inch
elif cond_diam == 2:
    tests = tests_2inch

test_summary = ['X (cm),Y (cm), start(s), end(s), omega, stdev-Y (cm), stdev-omega, sensor used'] 

for test in tests:
    filepath = tests[test][1]
    x = int(test[1:3])
    obj = BinaryFileUnpack(filepath)
    arr = tests[test][0]
    for i in range(len(arr)):
        # Get parameters for oscillation from dictionary
        start_time = arr[i][0]
        end_time = arr[i][1]
        sens_ind = arr[i][2]
        t = end_time - start_time
        # Index position for start and end times
        s = int(start_time * obj.fs)
        e = int(end_time * obj.fs)
        
        # Filter the high-frequency noise from data
        P = obj.P[sens_ind, s:e]
        bound_freq = 2.2
        nyq_freq = obj.fs // 2
        b, a = signal.butter(1, bound_freq/nyq_freq, 'lowpass')
        filteredP = signal.filtfilt(b, a, P)
        # Get the period
        omega, omega_stdev = get_omega(start_time, end_time, filteredP, obj)
        
        # Determine base pressures with provided sensor index
        # Determine H with provided conduit diameter
        pres, pres_std, H = spec_params(cond_diam, sens_ind, path, x)
        
        # Obtain average h_y values for experiment
        y = (np.mean(filteredP) - pres) * 1e5/98
        y_stddev = 1e5/98 * np.sqrt(np.var(filteredP) + pres_std**2)

        # Convert to X and Y as defined in Max's illustration
        # X = x - (Hb - H)
        # Y = y + (H + t)
        test_summary.append(f"{x-(Hb-H)}, {y+(H+t)}, {start_time}, {end_time}, {omega}, {y_stddev}, {omega_stdev}, {sens_ind+1}")

for data in test_summary:
    print(data)

Put Extracted Data in .csv file

In [ ]:
# Writing to .csv file
# Write initial data
csv_name = f'hot-water-freq-filtered-{int(cond_diam)}in.csv'
with open(csv_name, 'w') as file_write:
    for test in test_summary:
        file_write.write(test + '\n')